In [1]:
import os
import librosa
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile
from utils.evaluate import edit_distance
from utils.utils import cut_audio, segmentAudio, peakLevelAudio, SpectralGate, Speech2TextAPI
from utils.agc import AGCModel, Expect, Maximize

# Preprocess

In [2]:
filenames = ['test_male1', 'test_male2', 'test_male3',
            'test_female1', 'test_female2', 'test_female3',
            'voice_command1', 'voice_command1_volume'
            # '001_sd', '002_sd'
            ]

base_path = 'shortAudio' # shortAudio
folders = ['original', 'spectral-gate', 'spectral-gate-volume', 'spectral-gate-tune', 'automatic-gain-control', 'agcsg', 'sgagc']
for folder in folders:
  print(folder)
  for fn in tqdm(filenames, desc="Prepare Directory"):
    origin_path = os.path.join(base_path, folder)

    path = os.path.join(origin_path, fn)
    if not os.path.isdir(path):
      print("Make Directory:", path)
      os.mkdir(path)

original


Prepare Directory: 100%|██████████| 8/8 [00:00<?, ?it/s]


spectral-gate


Prepare Directory: 100%|██████████| 8/8 [00:00<00:00, 8012.04it/s]


spectral-gate-volume


...

Prepare Directory: 100%|██████████| 8/8 [00:00<00:00, 7973.96it/s]


agcsg


Prepare Directory: 100%|██████████| 8/8 [00:00<?, ?it/s]


sgagc


Prepare Directory: 100%|██████████| 8/8 [00:00<?, ?it/s]


## Cut audio files

In [21]:
second = 5
for fn in filenames:
  audio_path = os.path.join('11-2-22', fn + '.mp3')
  output_path = os.path.join(os.path.join('shortAudio', 'original'), fn)
  output_file = cut_audio(audio_path, output_path, second)

  # for out_f in output_file:
  #   output_path = os.path.join(os.path.join('shortAudio', 'spectral-gate'), fn)
  #   SpectralGate(out_f, output_path)

Audio Filepath: 11-2-22\test_male1.mp3
Length: 34.76897959183673 second


Cutting Process: 100%|██████████| 7/7 [00:00<00:00, 2358.05it/s]


Audio Filepath: 11-2-22\test_male2.mp3
Length: 29.15265306122449 second


Cutting Process: 100%|██████████| 6/6 [00:00<00:00, 1999.19it/s]


Audio Filepath: 11-2-22\test_male3.mp3
Length: 43.12816326530612 second


...

Cutting Process: 100%|██████████| 7/7 [00:00<00:00, 1749.61it/s]


Audio Filepath: 11-2-22\voice_command1.mp3
Length: 385.72802721088436 second


Cutting Process: 100%|██████████| 78/78 [00:00<00:00, 1813.83it/s]


Audio Filepath: 11-2-22\voice_command1_volume.mp3
Length: 385.72802721088436 second


Cutting Process: 100%|██████████| 78/78 [00:00<00:00, 1444.43it/s]


## Spectral Gating

In [61]:
for fn in filenames:
  origin_path = os.path.join(os.path.join('shortAudio', 'original'), fn)
  origin_filenames = np.array(os.listdir(origin_path))
  mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), origin_filenames))))

  for origin_f in origin_filenames[mask]:
    audio_path = os.path.join(origin_path, origin_f)
    output_path = os.path.join(os.path.join('shortAudio', 'spectral-gate'), fn)
    SpectralGate(audio_path, output_path)

## Spectral Gating With Increasing Volume

In [62]:
for fn in filenames:
  origin_path = os.path.join(os.path.join('shortAudio', 'original'), fn)
  origin_filenames = np.array(os.listdir(origin_path))
  mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), origin_filenames))))

  for origin_f in origin_filenames[mask]:
    audio_path = os.path.join(origin_path, origin_f)
    output_path = os.path.join(os.path.join('shortAudio', 'spectral-gate-volume'), fn)
    SpectralGate(audio_path, output_path, increase_volume=True)

## Automatic Gain Control

In [63]:
for fn in filenames:
  origin_path = os.path.join(os.path.join('shortAudio', 'original'), fn)
  origin_filenames = np.array(os.listdir(origin_path))
  mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), origin_filenames))))
  model = AGCModel(Expect(), Maximize())
  for origin_f in origin_filenames[mask]:
    audio_path = os.path.join(origin_path, origin_f)
    audio, sr = librosa.load(audio_path)
    audio_li = segmentAudio(audio, sr, 0.1)
    l = peakLevelAudio(audio, sr, 0.1)
    if model.mu is None or model.sigma is None:
      model.init_param(np.abs(audio))
    gain = model.run(l)
    up_scale_audio_li = model.scaleUp(gain, audio_li)
    output_path = os.path.join(os.path.join('shortAudio', 'automatic-gain-control'), fn)
    output_path = os.path.join(output_path, origin_f)
    wavfile.write(output_path, sr, np.concatenate(up_scale_audio_li).astype('f4'))

## Automatic Gain Control After Spectal Gating (agcsg)

In [67]:
for fn in filenames:
  origin_path = os.path.join(os.path.join('shortAudio', 'spectral-gate-tune'), fn)
  audio_original_basepath = os.path.join(os.path.join('shortAudio', 'original'), fn)
  origin_filenames = np.array(os.listdir(origin_path))
  mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), origin_filenames))))
  model = AGCModel(Expect(), Maximize())
  for origin_f in origin_filenames[mask]:
    audio_original_path = os.path.join(audio_original_basepath, origin_f)
    audio_spectral_path = os.path.join(origin_path, origin_f)
    audio_original, sr_original = librosa.load(audio_original_path)
    audio_spectral, sr_spectral = librosa.load(audio_spectral_path)

    audio_original_li = segmentAudio(audio_original, sr_original, 0.1)
    l_original = peakLevelAudio(audio_original, sr_original, 0.1)
    audio_spectral_li = segmentAudio(audio_spectral, sr_spectral, 0.1)
    # l = peakLevelAudio(audio, sr, 0.1)
    if model.mu is None or model.sigma is None:
      model.init_param(np.abs(audio_original))
    gain = model.run(l_original)
    up_scale_audio_li = model.scaleUp(gain, audio_spectral_li)
    output_path = os.path.join(os.path.join('shortAudio', 'agcsg'), fn)
    output_path = os.path.join(output_path, origin_f)
    wavfile.write(output_path, sr_spectral, np.concatenate(up_scale_audio_li).astype('f4'))

## Spectral Gating After Automatic Gain Control (sgagc)

In [65]:
for fn in filenames:
  origin_path = os.path.join(os.path.join('shortAudio', 'automatic-gain-control'), fn)
  origin_filenames = np.array(os.listdir(origin_path))
  mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), origin_filenames))))
  for origin_f in origin_filenames[mask]:
    audio_path = os.path.join(origin_path, origin_f)
    output_path = os.path.join(os.path.join('shortAudio', 'sgagc'), fn)
    SpectralGate(audio_path, output_path, increase_volume=False)

## Concate Short Audio to a long one

In [20]:
audio_li = []
base_path = 'shortAudio/original/voice_command1_volume'
filenames = np.array(os.listdir(base_path))
mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), filenames))))
for fn in filenames[mask]:
  audio, sr = librosa.load(os.path.join(base_path, fn))
  audio_li.append(audio)
new_audio = np.concatenate(audio_li)

In [21]:
wavfile.write('original_voice_command.mp3', sr, new_audio.astype('f4'))

# API

## Send Single Request

In [22]:
filename = 'test_male1'
idx = 3
base_data = {
    "audioData":None,
    "decoder_type": "BeamSearch",
    "get_word_timestamps": True,
    "word_list": ["ซี่"]
}

mode = 'original'
filepath = f'shortAudio/{mode}/{filename}/{filename}_{idx}.wav'
original_response = Speech2TextAPI(filepath, base_data=base_data)
print("Original:", original_response['results'][0]['transcript'])

mode = 'spectral-gate-volume'
filepath = f'shortAudio/{mode}/{filename}/{filename}_{idx}.wav'
reduced_response = Speech2TextAPI(filepath, base_data=base_data)
print("Reduce Noise:", reduced_response['results'][0]['transcript'])

Original: ห้าลบหนึ่งสามสี่หนึ่ง
Reduce Noise: ห้าลบหนึ่งสามสี่หนึ่ง


## Send Multiple Request

In [2]:
base_data = {
    "audioData":None,
    "decoder_type": "BeamSearch",
    "get_word_timestamps": True,
    # "word_list": ["ซี่"]
}

modes = ['agcsg', 'automatic-gain-control', 'original', 'sgagc', 'spectral-gate', 'spectral-gate-volume', 'spectral-gate-tune']
# idx = 3
for idx in range(7):
    mode = modes[idx]
    print(mode)
    filenames =  ['test_male1', 'test_male2', 'test_male3', 'test_female1', 'test_female2', 'test_female3', 'voice_command1', 'voice_command1_volume'] # ['voice_command1_volume']
    for f in filenames:
        basepath = f'shortAudio/{mode}/{f}'
        predicted = []
        ff = np.array(os.listdir(basepath))
        mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), ff))))
        for fn in tqdm(ff[mask]):
            filepath = os.path.join(basepath, fn)
            # print(filepath)
            try:
                response = Speech2TextAPI(filepath, base_data=base_data)
                text = ''.join(list(map(lambda x: x['transcript'], response['results'])))
            except:
                text = 'Internal Server Error: 0'
            predicted.append(text)
            # print("Predict:", text)
        df_label = pd.read_csv(f'label/{f}.csv')
        data = {'Label': df_label['Label'], 'Predicted': predicted}
        df = pd.DataFrame(data)
        df.to_csv(f'predicted/{mode}/{f}.csv', index=False)
        print(f'predicted/{mode}/{f}.csv')
        print()

agcsg


100%|██████████| 7/7 [00:10<00:00,  1.46s/it]


predicted/agcsg/test_male1.csv



100%|██████████| 6/6 [00:08<00:00,  1.38s/it]


predicted/agcsg/test_male2.csv



...

predicted/spectral-gate-tune/test_female3.csv



100%|██████████| 78/78 [02:26<00:00,  1.87s/it]


predicted/spectral-gate-tune/voice_command1.csv



100%|██████████| 78/78 [02:21<00:00,  1.81s/it]

predicted/spectral-gate-tune/voice_command1_volume.csv



# Evaluate

## New

In [22]:
basepath = 'predicted'
folder = ['original', 'spectral-gate', 'spectral-gate-volume', 'spectral-gate-tune', 'automatic-gain-control', 'agcsg', 'sgagc']
edit_df = []

for f in folder:
  path = os.path.join(basepath, f)
  # filenames = ['test_male1.csv', 'test_male2.csv', 'test_male3.csv', 'test_female1.csv', 'test_female2.csv', 'test_female3.csv']
  filenames = ['voice_command1.csv'] # os.listdir(path)
  li = []
  for fn in filenames:
    # print(fn)
    filepath = os.path.join(path, fn)
    df = pd.read_csv(filepath)
    # display(df)
    mask = df['Predicted'] == 'Internal Server Error: 0'
    df = df[~mask]
    df['Edit Distance'] = df.apply(lambda x: edit_distance(x['Label'], x['Predicted']), axis=1) 
    if fn == 'test_female1.csv':
      try:
        df = df.drop(labels=[0, 1], axis=0)
      except:
        pass
    elif fn == 'test_male3.csv':
      try:
        df = df.drop(labels=[0], axis=0)
      except:
        pass
    li.append(df)
  df = pd.concat(li, keys=filenames)
  edit_df.append(df)

In [23]:
for idx, mode in enumerate(folder):
  print(f"Edit Distance of {mode}:", round(edit_df[idx]['Edit Distance'].mean(), 2), '+-', round(edit_df[idx]['Edit Distance'].std(), 2))
  print()

Edit Distance of original: 8.06 +- 4.95

Edit Distance of spectral-gate: 9.61 +- 4.3

Edit Distance of spectral-gate-volume: 5.29 +- 5.27

Edit Distance of spectral-gate-tune: 1.99 +- 4.3

Edit Distance of automatic-gain-control: 2.57 +- 4.3

Edit Distance of agcsg: 2.43 +- 4.14

Edit Distance of sgagc: 2.25 +- 4.17



## Old

In [28]:
basepath = 'label'
folder = ['male', 'female']
edit_df = []

for f in folder:
  path = os.path.join(basepath, f)
  filenames = os.listdir(path)
  li = []
  for fn in filenames:
    print(fn)
    filepath = os.path.join(path, fn)
    df = pd.read_csv(filepath)
    mask = np.logical_or(df['Speech-To-Text (Original)'] == 'Internal Server Error:0', 
                          df['Speech-To-Text (Reduce Noise)'] == 'Internal Server Error:0').values
    df = df[~mask]
    df['Edit Distance (Original)'] = df.apply(lambda x: edit_distance(x['Label'], x['Speech-To-Text (Original)']), axis=1) 
    df['Edit Distance (Reduce Noise)'] = df.apply(lambda x: edit_distance(x['Label'], x['Speech-To-Text (Reduce Noise)']), axis=1)
    li.append(df)
  df = pd.concat(li, keys=filenames)
  edit_df.append(df)

test_male_1.csv
test_male_2.csv
test_male_3.csv
test_female_1.csv
test_female_2.csv
test_female_3.csv


### Male

In [42]:
edit_df[0]

Time  \
test_male_1.csv 0  0:00 - 0:10   
                1  0:10 - 0:20   
                2  0:20 - 0:30   
                3  0:30 - 0:35   
test_male_2.csv 0  0:00 - 0:10   
                1  0:10 - 0:20   
                2  0:20 - 0:29   
test_male_3.csv 0  0:00 - 0:10   
                1  0:10 - 0:20   
                2  0:20 - 0:30   
                3  0:30 - 0:40   
                4  0:40 - 0:43   

                                                               Label  \
test_male_1.csv 0    ซี่หนึ่งเจ็ด ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า ซี่   
                1    หนึ่งหก ลบหนึ่งสี่ ลบสองห้า ลบหนึ่งสาม ซี่หนึ่ง   
                2       ห้า ลบสองสาม ลบหนึ่งห้า ลบสามสี่ ซี่หนึ่งสี่   
                3                                 ลบหนึ่งหก ลบสองห้า   
test_male_2.csv 0   เริ่ม ซี่หนึ่งสี่ ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า   
                1  แก้ใหม่ซี่หนึ่งสี่ ลบหนึ่งสี่ ลบสองสาม ลบหนึ่ง...   
                2                ลบสองห้า ลบหนึ่งสี่ ลบสองห้า ครับผม   
test_male_3.csv 0  รบกวนอาจารย์อีกรอบนะครับ ได้ครับผม เออ ซี่ ซี่...   
                1  ลบหนึ่งสาม ลบสองหก ลบสองห้า ซี่หนึ่งหก ลบหนึ่งสี่   
                2       ลบสองสาม ลบสามสี่ ซี่หนึ่งห้า ลบห้าสาม ลบสอง   
                3    สี่ ลบสามสาม แก้ไขซี่หนึ่งห้า ลบสองห้า ลบสองสาม   
                4                                         ลบหนึ่งสี่   

                                           Speech-To-Text (Original)  \
test_male_1.csv 0  Cี่หนึ่งเจ็ด ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า สี...   
                1    หนึ่งหก ลบหนึ่งสี่ ลบสองห้า ลบหนึ่งสาม สี่หนึ่ง   
                2       ห้า ลบสองสาม ลบหนึ่งห้า ลบสามสี่ สี่หนึ่งสี่   
                3                                 ลบหนึ่งหก ลบสองห้า   
test_male_2.csv 0  นครับ cี่หนึ่งสี่ ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า้   
                1        หลบหนึ่งสี่ ลบสองสาม ลบหนึ่งห้า สี่หนึ่งสาม   
                2                 สาลบสองห้า ลบหนึ่งสี่ ลบสองห้า ครั   
test_male_3.csv 0               คอาการย ก็พอ ได้ครับ ซี่ สีหนึ่งเจ็ด   
                1  ลบหนึ่งสาม ลบสองหก ลบสองห้า สี่หนึ่งหก ลบหนึ่งสี่   
                2       ลบสองสาม ลบสามสี่ สี่หนึ่งห้า ลบห้าสาม ลบสอง   
                3    สี่ ลบสามสาม แก้ไขสี่หนึ่งห้า ลบสองห้า ลบสองสาม   
                4                                         ลบหนึ่งสี่   

                                       Speech-To-Text (Reduce Noise)  \
test_male_1.csv 0    ซี่หนึ่งเจ็ด ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า สิน   
                1  หนึ่งหก ลบหนึ่งสี่ ลบสองห้า ลบหนึ่งสาม สี่หนึ่...   
                2      ค่าค่ ลบสองสาม ลบหนึ่งห้า ลบสามสี่ ซีหนึ่งสี่   
                3                                 ลบหนึ่งหก ลบสองห้า   
test_male_2.csv 0  เลือก สี่หนึ่งสี่ ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห...   
                1  ใหม่ซีหนึ่งสี่ ลบหนึ่งสิ่ง ลบสองสาม พบหนึ่งห้า...   
                2              ถาม ลบสองห้า ลบหนึ่งสี่ ลบสองห้า ครับ   
test_male_3.csv 0  คณอาการย เพราะพอได้ ข้าหกคนคนหนึ่ง อ่ซี่ สีหนึ...   
                1  ลบหนึ่งสาม ลบสองหก ซุดสองห้า วี่หนึ่งหก ลบหนึ่...   
                2      ลบสองสาม พบสายจริง ที่หนึ่งห้า ลบห้าสาม ลบสอง   
                3    สี่ ลบสามสาม แก้ไขที่หนึ่งห้า ลบสองห้า ลบสองสาม   
                4                                         ลบหนึ่งสี่   

                   Edit Distance (Original)  Edit Distance (Reduce Noise)  
test_male_1.csv 0                       7.0                           3.0  
                1                       1.0                           4.0  
                2                       1.0                           5.0  
                3                       0.0                           0.0  
test_male_2.csv 0                       7.0                           9.0  
                1                      19.0                           7.0  
                2                       5.0                           6.0  
test_male_3.csv 0                      26.0                          35.0  
                1                       1.0                           4.0  
            

In [35]:
print("Edit Distance of Male:")
print()
print(edit_df[0].mean(numeric_only=True))

Edit Distance of Male:

Edit Distance (Original)        5.75
Edit Distance (Reduce Noise)    6.75
dtype: float64


### Female

In [43]:
edit_df[1]

Time  \
test_female_1.csv 1  0:10 - 0:20   
                  2  0:20 - 0:30   
                  3  0:30 - 0:39   
test_female_2.csv 0  0:00 - 0:10   
                  1  0:10 - 0:20   
                  2  0:20 - 0:23   
test_female_3.csv 0  0:00 - 0:10   
                  1  0:10 - 0:20   
                  2  0:20 - 0:30   

                                                                 Label  \
test_female_1.csv 1                  หนึ่งสอง สองสี่ สี่ห้า ห้า สองห้า   
                  2         หนึ่งสอง หนึ่งสี่ สามสอง หนึ่งสอง หนึ่งสอง   
                  3  เริ่มหนึ่งห้า สองสอง หนึ่งสอง ศูนย์หนึ่ง อาจจะลอง   
test_female_2.csv 0  เชิญครับ ค่ะ เริ่มหนึ่งหนึ่งดิททรัล หนึ่งสอง ห...   
                  1   หนึ่งสี่ สองสี่ ขอแก้ไข สองหนึ่ง หนึ่งสอง สี่สอง   
                  2                              สามสอง ค่ะ ขอบคุณครับ   
test_female_3.csv 0     หนึ่งแปดดิททรัล ลบสองหนึ่ง ลบสองสาม ศูนย์หนึ่ง   
                  1       สองสี่ สองสอง ศูนย์หนึ่ง แก้ไข ใหม่ ซี่หนึ่ง   
                  2      เจ็ด ดิททรัล สอง ลบสองหนึ่ง ลบสองสอง ศูนย์ห้า   

                                           Speech-To-Text (Original)  \
test_female_1.csv 1                หนึ่งสอง สองสี่ สี่ห้า ห้า สองห้า   
                  2    หนึ่งสอง หนึ่งสี่ ครบสามสอง หนึ่งสอง หนึ่งสอง   
                  3         รรนห สอสอง หนึ่งสอง ศูนย์หนึ่ง อาจจะล้อง   
test_female_2.csv 0           เริ่มหนึ่งหนึ่งดิตอล หนึ่งสอง หนึ่งสอง   
                  1  หนึ่งสี่ สองสี่ สแก้ สองหนึ่ง หนึ่งสองคะ สี่สอง   
                  2                                        สสามสอง ค   
test_female_3.csv 0        หนึ่งแปดติตอล ลบสองนึ่ง ลบสองส ศูนย์หนึ่ง   
                  1             สองสี่ สองสอง ศูนย์หนึ่ง แก้ไข หม น่   
                  2                      น สอง ลสองนง ลบสองสอง ศนห้า   

                                        Speech-To-Text (Reduce Noise)  \
test_female_1.csv 1                 หนึ่งสอง สองสี่ สี่ห้า ห้า สองห้า   
                  2        หนึ่งสอง หนึ่งสี่ สามสอง หนึ่งสอง หนึ่งสอง   
                  3   ร่มหนึ่งห้า สองสอง หนึ่งสอง ศูนย์หนึ่ง อาจจะลอง   
test_female_2.csv 0            เริ่มหนึ่งหนึ่งดิตอล หนึ่งสอง หนึ่งสอง   
                  1  หนึ่งสี่ สองสี่ ขอแก้ไข สองหนึ่ง หนึ่งสอง สี่จอง   
                  2                              สองสาม สองฉักอีกแล้ว   
test_female_3.csv 0   หนึ่งแปดดิซซอล ลบสองหนึ่ง แล้วสองสาม ศูนย์หนึ่ง   
                  1   สองสี่ สองสอง ศูนย์หนึ่ง แก้ไขใหม่ ที่หนึ่งเส็ด   
                  2        net ดิตอล สอง ลบสองหนึ่ง ลดสองสอง ศูนย์ห้า   

                     Edit Distance (Original)  Edit Distance (Reduce Noise)  
test_female_1.csv 1                       0.0                           0.0  
                  2                       3.0                           0.0  
                  3                      12.0                           2.0  
test_female_2.csv 0                      17.0                          17.0  
                  1                       6.0                           1.0  
                  2                      14.0                          18.0  
test_female_3.csv 0                       8.0                           7.0  
                  1                       8.0                           6.0  
                  2                      19.0                           9.0

In [36]:
print("Edit Distance of Female:")
print()
print(edit_df[1].mean(numeric_only=True))

Edit Distance of Female:

Edit Distance (Original)        9.666667
Edit Distance (Reduce Noise)    6.666667
dtype: float64


### All

In [37]:
all_df = pd.concat(edit_df)

In [44]:
all_df

Time  \
test_male_1.csv   0  0:00 - 0:10   
                  1  0:10 - 0:20   
                  2  0:20 - 0:30   
                  3  0:30 - 0:35   
test_male_2.csv   0  0:00 - 0:10   
                  1  0:10 - 0:20   
                  2  0:20 - 0:29   
test_male_3.csv   0  0:00 - 0:10   
                  1  0:10 - 0:20   
                  2  0:20 - 0:30   
                  3  0:30 - 0:40   
                  4  0:40 - 0:43   
test_female_1.csv 1  0:10 - 0:20   
                  2  0:20 - 0:30   
                  3  0:30 - 0:39   
test_female_2.csv 0  0:00 - 0:10   
                  1  0:10 - 0:20   
                  2  0:20 - 0:23   
test_female_3.csv 0  0:00 - 0:10   
                  1  0:10 - 0:20   
                  2  0:20 - 0:30   

                                                                 Label  \
test_male_1.csv   0    ซี่หนึ่งเจ็ด ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า ซี่   
                  1    หนึ่งหก ลบหนึ่งสี่ ลบสองห้า ลบหนึ่งสาม ซี่หนึ่ง   
                  2       ห้า ลบสองสาม ลบหนึ่งห้า ลบสามสี่ ซี่หนึ่งสี่   
                  3                                 ลบหนึ่งหก ลบสองห้า   
test_male_2.csv   0   เริ่ม ซี่หนึ่งสี่ ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า   
                  1  แก้ใหม่ซี่หนึ่งสี่ ลบหนึ่งสี่ ลบสองสาม ลบหนึ่ง...   
                  2                ลบสองห้า ลบหนึ่งสี่ ลบสองห้า ครับผม   
test_male_3.csv   0  รบกวนอาจารย์อีกรอบนะครับ ได้ครับผม เออ ซี่ ซี่...   
                  1  ลบหนึ่งสาม ลบสองหก ลบสองห้า ซี่หนึ่งหก ลบหนึ่งสี่   
                  2       ลบสองสาม ลบสามสี่ ซี่หนึ่งห้า ลบห้าสาม ลบสอง   
                  3    สี่ ลบสามสาม แก้ไขซี่หนึ่งห้า ลบสองห้า ลบสองสาม   
                  4                                         ลบหนึ่งสี่   
test_female_1.csv 1                  หนึ่งสอง สองสี่ สี่ห้า ห้า สองห้า   
                  2         หนึ่งสอง หนึ่งสี่ สามสอง หนึ่งสอง หนึ่งสอง   
                  3  เริ่มหนึ่งห้า สองสอง หนึ่งสอง ศูนย์หนึ่ง อาจจะลอง   
test_female_2.csv 0  เชิญครับ ค่ะ เริ่มหนึ่งหนึ่งดิททรัล หนึ่งสอง ห...   
                  1   หนึ่งสี่ สองสี่ ขอแก้ไข สองหนึ่ง หนึ่งสอง สี่สอง   
                  2                              สามสอง ค่ะ ขอบคุณครับ   
test_female_3.csv 0     หนึ่งแปดดิททรัล ลบสองหนึ่ง ลบสองสาม ศูนย์หนึ่ง   
                  1       สองสี่ สองสอง ศูนย์หนึ่ง แก้ไข ใหม่ ซี่หนึ่ง   
                  2      เจ็ด ดิททรัล สอง ลบสองหนึ่ง ลบสองสอง ศูนย์ห้า   

                                             Speech-To-Text (Original)  \
test_male_1.csv   0  Cี่หนึ่งเจ็ด ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า สี...   
                  1    หนึ่งหก ลบหนึ่งสี่ ลบสองห้า ลบหนึ่งสาม สี่หนึ่ง   
                  2       ห้า ลบสองสาม ลบหนึ่งห้า ลบสามสี่ สี่หนึ่งสี่   
                  3                                 ลบหนึ่งหก ลบสองห้า   
test_male_2.csv   0  นครับ cี่หนึ่งสี่ ลบหนึ่งสาม ลบสองสี่ ลบหนึ่งห้า้   
                  1        หลบหนึ่งสี่ ลบสองสาม ลบหนึ่งห้า สี่หนึ่งสาม   
                  2                 สาลบสองห้า ลบหนึ่งสี่ ลบสองห้า ครั   
test_male_3.csv   0               คอาการย ก็พอ ได้ครับ ซี่ สีหนึ่งเจ็ด   
                  1  ลบหนึ่งสาม ลบสองหก ลบสองห้า สี่หนึ่งหก ลบหนึ่งสี่   
                  2       ลบสองสาม ลบสามสี่ สี่หนึ่งห้า ลบห้าสาม ลบสอง   
                  3    สี่ ลบสามสาม แก้ไขสี่หนึ่งห้า ลบสองห้า ลบสองสาม   
                  4                                         ลบหนึ่งสี่   
test_female_1.csv 1                  หนึ่งสอง สองสี่ สี่ห้า ห้า สองห้า   
                  2      หนึ่งสอง หนึ่งสี่ ครบสามสอง หนึ่งสอง หนึ่งสอง   
                  3           รรนห สอสอง หนึ่งสอง ศูนย์หนึ่ง อาจจะล้อง   
test_female_2.csv 0             เริ่มหนึ่งหนึ่งดิตอล หนึ่งสอง หนึ่งสอง   
                  1    หนึ่งสี่ สองสี่ สแก้ สองหนึ่ง หนึ่งสองคะ สี่สอง   
                  2                                          สสามสอง ค   
test_female_3.csv 0          หนึ่งแปดติตอล ลบสองนึ่ง ลบสองส ศูนย์หนึ่ง   
                  1               สองสี่ สองสอง ศูนย์หนึ่ง แก้ไข หม น่   
                  2                        น สอง ลสอ

In [40]:
print("Edit Distance:")
print()
print(all_df.mean(numeric_only=True))

Edit Distance:

Edit Distance (Original)        7.428571
Edit Distance (Reduce Noise)    6.714286
dtype: float64


In [25]:
import requests
import json
import base64
with open('spectral-gate/test_female1.wav', 'rb') as fh:
  content = fh.read()

res = requests.post(
  'https://gowajee-api-mgmt.azure-api.net/speech/transcribe',
  data=json.dumps(
    {"audioData":base64.encodebytes(content).decode('utf-8')
    }),
  headers={
    'Content-type': 'application/json',
    'x-api-key':'8yYrTF/Ya97OBpRaKok9+pBeBjkXK6Wkl0j4dPHdrCs='
  }
) 

In [26]:
res

<Response [413]>

In [27]:
res.json()

JSONDecodeError: [Errno Expecting value] <html>
<head><title>413 Request Entity Too Large</title></head>
<body>
<center><h1>413 Request Entity Too Large</h1></center>
<hr><center>nginx</center>
</body>
</html>
: 0

## Fine Tune Spectral Gating

In [55]:
filenames = ['test_male1', 'test_male2', 'test_male3', 'test_female1', 'test_female2', 'test_female3']
config_space = {
  'stationary': [True, False], 
  'time_constant_s': [1, 1.5, 2],
  'freq_mask_smooth_hz': [250, 500, 750],
  'time_mask_smooth_ms': [25, 50, 75],
  'thresh_n_mult_nonstationary': [0.5, 1, 1.5],
  'sigmoid_slope_nonstationary': [7, 10, 13],
  'n_fft': [512, 1024]
}

for iter in range(10):
  config = dict(map(lambda kv: (kv[0], random.choice(kv[1])), config_space.items()))
  print("Iter:", iter)
  print(config)
  filenames = ['test_male1', 'test_male2', 'test_male3', 'test_female1', 'test_female2', 'test_female3']
  for fn in filenames:
    origin_path = os.path.join(os.path.join('shortAudio', 'original'), fn)
    origin_filenames = os.listdir(origin_path)

    for origin_f in origin_filenames:
      audio_path = os.path.join(origin_path, origin_f)
      output_path = os.path.join(os.path.join('shortAudio', 'spectral-gate-tune'), fn)
      SpectralGate(audio_path, output_path, increase_volume=True, config=config)

  modes = ['spectral-gate-tune']
  idx = 0
  mode = modes[idx]
  
  for f in filenames:
      basepath = f'shortAudio/{mode}/{f}'
      predicted = []
      for fn in os.listdir(basepath):
          filepath = os.path.join(basepath, fn)
          try:
              response = Speech2TextAPI(filepath)
              text = response['results'][0]['transcript']
          except:
              text = response['results'][0]['transcript']
          predicted.append(text)
      df_label = pd.read_csv(f'label/{f}.csv')
      data = {'Label': df_label['Label'], 'Predicted': predicted}
      df = pd.DataFrame(data)
      df.to_csv(f'predicted/{mode}/{f}.csv', index=False)

  basepath = 'predicted'
  folder = ['spectral-gate-volume', 'spectral-gate-tune']
  edit_df = []

  for f in folder:
    path = os.path.join(basepath, f)
    filenames = os.listdir(path)
    li = []
    for fn in filenames:
      # print(fn)
      filepath = os.path.join(path, fn)
      df = pd.read_csv(filepath)
      mask = df['Predicted'] == 'Internal Server Error: 0'
      df = df[~mask]
      df['Edit Distance'] = df.apply(lambda x: edit_distance(x['Label'], x['Predicted']), axis=1) 
      if fn == 'test_female1.csv':
        try:
          df = df.drop(labels=[0, 1], axis=0)
        except:
          pass
      elif fn == 'test_male3.csv':
        try:
          df = df.drop(labels=[0], axis=0)
        except:
          pass
      li.append(df)
    df = pd.concat(li, keys=filenames)
    edit_df.append(df)

  for idx, mode in enumerate(folder):
    print(f"Edit Distance of {mode}:", edit_df[idx]['Edit Distance'].mean())

  print()

Iter: 0
{'stationary': True, 'time_constant_s': 1.5, 'freq_mask_smooth_hz': 500, 'time_mask_smooth_ms': 75, 'thresh_n_mult_nonstationary': 0.5, 'sigmoid_slope_nonstationary': 7, 'n_fft': 1024}
Edit Distance of spectral-gate-volume: 10.256410256410257
Edit Distance of spectral-gate-tune: 12.820512820512821

Iter: 1
{'stationary': True, 'time_constant_s': 1.5, 'freq_mask_smooth_hz': 250, 'time_mask_smooth_ms': 25, 'thresh_n_mult_nonstationary': 1.5, 'sigmoid_slope_nonstationary': 7, 'n_fft': 512}
Edit Distance of spectral-gate-volume: 10.256410256410257
Edit Distance of spectral-gate-tune: 11.58974358974359

Iter: 2
{'stationary': False, 'time_constant_s': 2, 'freq_mask_smooth_hz': 750, 'time_mask_smooth_ms': 25, 'thresh_n_mult_nonstationary': 1, 'sigmoid_slope_nonstationary': 7, 'n_fft': 512}
Edit Distance of spectral-gate-volume: 10.256410256410257
Edit Distance of spectral-gate-tune: 7.871794871794871

Iter: 3
{'stationary': True, 'time_constant_s': 2, 'freq_mask_smooth_hz': 250, 'tim

In [33]:
best_config = {
  'stationary': False, 
  'time_constant_s': 2, 
  'freq_mask_smooth_hz': 250, 
  'time_mask_smooth_ms': 50, 
  'thresh_n_mult_nonstationary': 0.5, 
  'sigmoid_slope_nonstationary': 10, 
  'n_fft': 1024
}

In [66]:
for fn in filenames:
  origin_path = os.path.join(os.path.join('shortAudio', 'original'), fn)
  origin_filenames = np.array(os.listdir(origin_path))
  mask = np.argsort(np.array(list(map(lambda x: int(x.split('_')[-1][:-4]), origin_filenames))))

  for origin_f in origin_filenames:
    audio_path = os.path.join(origin_path, origin_f)
    output_path = os.path.join(os.path.join('shortAudio', 'spectral-gate-tune'), fn)
    SpectralGate(audio_path, output_path, increase_volume=True, config=best_config)

In [9]:
config = {
  'stationary': False, 
  'time_constant_s': 2, 
  'freq_mask_smooth_hz': 250, 
  'time_mask_smooth_ms': 50, 
  'thresh_n_mult_nonstationary': 0.5, 
  'sigmoid_slope_nonstationary': 10, 
  'n_fft': 1024
}

print(config)
filenames = ['test_male1', 'test_male2', 'test_male3', 'test_female1', 'test_female2', 'test_female3', 'voice_command1']
for fn in filenames:
  origin_path = os.path.join(os.path.join('shortAudio', 'original'), fn)
  origin_filenames = os.listdir(origin_path)

  for origin_f in origin_filenames:
    audio_path = os.path.join(origin_path, origin_f)
    output_path = os.path.join(os.path.join('shortAudio', 'spectral-gate-tune'), fn)
    SpectralGate(audio_path, output_path, increase_volume=True, config=config)

modes = ['spectral-gate-tune']
idx = 0
mode = modes[idx]

for f in filenames:
    basepath = f'shortAudio/{mode}/{f}'
    predicted = []
    for fn in os.listdir(basepath):
        filepath = os.path.join(basepath, fn)
        try:
            response = Speech2TextAPI(filepath)
            text = response['results'][0]['transcript']
        except:
            text = response['results'][0]['transcript']
        predicted.append(text)
    df_label = pd.read_csv(f'label/{f}.csv')
    data = {'Label': df_label['Label'], 'Predicted': predicted}
    df = pd.DataFrame(data)
    df.to_csv(f'predicted/{mode}/{f}.csv', index=False)

basepath = 'predicted'
folder = ['spectral-gate-volume', 'spectral-gate-tune']
edit_df = []

for f in folder:
  path = os.path.join(basepath, f)
  filenames = os.listdir(path)
  li = []
  for fn in filenames:
    # print(fn)
    filepath = os.path.join(path, fn)
    df = pd.read_csv(filepath)
    mask = df['Predicted'] == 'Internal Server Error: 0'
    df = df[~mask]
    df['Edit Distance'] = df.apply(lambda x: edit_distance(x['Label'], x['Predicted']), axis=1) 
    if fn == 'test_female1.csv':
      try:
        df = df.drop(labels=[0, 1], axis=0)
      except:
        pass
    elif fn == 'test_male3.csv':
      try:
        df = df.drop(labels=[0], axis=0)
      except:
        pass
    li.append(df)
  df = pd.concat(li, keys=filenames)
  edit_df.append(df)

for idx, mode in enumerate(folder):
  print(f"Edit Distance of {mode}:", edit_df[idx]['Edit Distance'].mean())

print()

{'stationary': False, 'time_constant_s': 2, 'freq_mask_smooth_hz': 250, 'time_mask_smooth_ms': 50, 'thresh_n_mult_nonstationary': 0.5, 'sigmoid_slope_nonstationary': 10, 'n_fft': 1024}


AttributeError: 'float' object has no attribute 'replace'